# Spotify Requests 

# Setup

* dotenv is needed to load the environment variables

In [1]:
%load_ext dotenv

In [2]:
%dotenv

In [3]:
import pandas as pd
import os
import numpy as np
import requests
import spotipy
from itertools import chain

## Load envs from a .env file

In [4]:
USERNAME = os.environ.get('USERNAME')
CLIENT_ID =os.environ.get('CLIENT_ID')
CLIENT_SECRET = os.environ.get('CLIENT_SECRET')
REDIRECT_URI = os.environ.get('REDIRECT_URI')

AUTH_URL = 'https://accounts.spotify.com/api/token'
SCOPE_READ = 'user-library-read'
SCOPE_TOP = 'user-top-read'
BASE_URL = 'https://api.spotify.com/v1/'

# Functions and methods

There are some functions that encapsulates the requests on the API, and other that processing the data received.

## Requests Functions

In [5]:
def get_token_scope(scope:str) -> str:
    return spotipy.util.prompt_for_user_token(username=USERNAME,
                                           scope=scope,
                                           client_id=CLIENT_ID,
                                           client_secret=CLIENT_SECRET,
                                           redirect_uri=REDIRECT_URI
                                           )

In [6]:
def get_headers(token) -> dict:
    return {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'Authorization': 'Bearer {token}'.format(token=token),
        }

In [7]:
def get_request(url:str, headers: dict, params: dict) -> dict:
    try:
        response = requests.get(url, 
                    headers = headers, params = params)
        json = response.json()
        first_result = json['items']
        return first_result
    except:
        return None

## Processing Functions

In [8]:
def get_attributes_from_artists(data: list):
    followers = []
    genres = []
    name = []
    popularity = []
    uri = []
    
    for index in range(len(data)):
        followers.append(data[index]['followers']['total'])
        genres.append(data[index]['genres'])
        name.append(data[index]['name'])
        popularity.append(data[index]['popularity'])
        uri.append(data[index]['uri'])
                   
    return uri, followers, genres, name, popularity                     

In [9]:
def create_df_artists(uri: list, names: list, genres: list, popularities: list, followers: list, columns_artist: list) -> pd.DataFrame:
    
    selected_data = {}
    selected_data[columns_artist[0]] = uri
    selected_data[columns_artist[1]] = names
    selected_data[columns_artist[2]] = genres
    selected_data[columns_artist[3]] = popularities
    selected_data[columns_artist[4]] = followers
    
    return pd.DataFrame.from_dict(selected_data)

In [31]:
def get_attributes_from_tracks(data: list):
    track_uri = []
    track_name = []
    duration = []
    explicit = []
    artist_name = []
    popularity = []
    album_name = []
    album_release_date = []
    album_total_tracks = []
    album_track_number = []
    available_markets = []
    preview_url = []
    
    for index in range(len(data)):
        track_uri.append(data[index]['uri'])
        track_name.append(data[index]['name'])
        duration.append(data[index]['duration_ms'])
        explicit.append(data[index]['explicit'])
        artist_name.append([k['name'] for k in list(chain(data[index]['artists']))])
        popularity.append(data[index]['popularity'])
        album_name.append(data[index]['album']['name'])
        album_release_date.append(data[index]['album']['release_date'])
        album_total_tracks.append(data[index]['album']['total_tracks'])
        album_track_number.append(data[index]['track_number'])
        available_markets.append(data[index]['available_markets'])
        preview_url.append(data[index]['preview_url'])
    
    return track_uri, track_name, duration, explicit, artist_name, popularity, album_name, album_release_date, album_total_tracks, album_track_number, available_markets, preview_url

In [32]:
def create_df_tracks(track_uri: list, track_name: list, duration: list, explicit: list, artist_name: list,
                    popularity: list, album_name: list, album_release_date: list, album_total_tracks:list, 
                    album_track_number: list, available_markets:list, preview_url:list) -> pd.DataFrame:
    
    selected_data = {}
    
    selected_data[columns_tracks[0]] = track_uri
    selected_data[columns_tracks[1]] = track_name
    selected_data[columns_tracks[2]] = duration
    selected_data[columns_tracks[3]] = explicit
    selected_data[columns_tracks[4]] = artist_name
    selected_data[columns_tracks[5]] = popularity
    selected_data[columns_tracks[6]] = album_name
    selected_data[columns_tracks[7]] = album_release_date
    selected_data[columns_tracks[8]] = album_total_tracks
    selected_data[columns_tracks[9]] = album_track_number
    selected_data[columns_tracks[10]] = available_markets
    selected_data[columns_tracks[11]] = preview_url
    

    return pd.DataFrame.from_dict(selected_data)

# Requests

In [12]:
token_saved = get_token_scope(SCOPE_READ)
token_top = get_token_scope(SCOPE_TOP)

In [13]:
headers_saved = get_headers(token_saved)
headers_top = get_headers(token_top)

In [14]:
saved_tracks_from_lib = get_request(BASE_URL+f'me/tracks', headers_saved, params = {'market': 'BR', 'limit':'50'})

In [15]:
top_tracks = get_request(BASE_URL+f'me/top/tracks', headers_top, params = {'time_range': 'medium_term', 'limit': '50'})

In [16]:
top_tracks_long =  get_request(BASE_URL+f'me/top/tracks', headers_top, params = {'time_range': 'long_term', 'limit':'50'})

In [17]:
top_artists = get_request(BASE_URL+f'me/top/artists', headers_top, params = {'time_range': 'medium_term', 'limit': '50'})

In [18]:
top_artists_long = get_request(BASE_URL+f'me/top/artists', headers_top, params = {'time_range': 'long_term', 'limit': '50'})

# Processing 

## Top artists

Every artist is placed as one element in the list returned. This element is a dictionary, and some of the keys are:
- genres
- images
- popularity
- name
- followers
- uri

In [19]:
columns_artist = ['uri', 'name', 'genres', 'popularity', 'followers']

In [20]:
uri_medium, followers_medium, genres_medium, names_medium, popularities_medium = get_attributes_from_artists(top_artists)

In [21]:
top_artists_medium = create_df_artists(uri_medium, names_medium, genres_medium, popularities_medium, followers_medium, columns_artist)
top_artists_medium

,uri,name,genres,popularity,followers
0,spotify:artist:5Pb27ujIyYb33zBqVysBkj,RÜFÜS DU SOL,"[australian electropop, indietronica]",73,838686
1,spotify:artist:3nFkdlSjzX9mRTtwJOzDYB,JAY-Z,"[east coast hip hop, hip hop, rap]",86,6622955
2,spotify:artist:0rasA5Z5h1ITtHelCpfu9R,Paul Kalkbrenner,"[electronica, german techno, leipzig electroni...",66,973581
3,spotify:artist:5wMlMjOLeJfS5DfxqGfm83,Jan Blomqvist,"[deep euro house, electronica]",58,159445
4,spotify:artist:6l3HvQ5sa6mXTsMTB19rO5,J. Cole,"[conscious hip hop, hip hop, north carolina hi...",89,14690059
5,spotify:artist:1jV311C5ADuBqCPpprsjUp,Klanglos,[dark techno],47,40102
6,spotify:artist:5K4W6rqBFWDnAN6FQUkS6x,Kanye West,"[chicago rap, rap]",96,15353943
7,spotify:artist:1CoZyIx7UvdxT5c8UkMzHd,Jorja Smith,"[pop, r&b, uk contemporary r&b]",76,2366675
8,spotify:artist:1GDbiv3spRmZ1XdM1jQbT7,Natti Natasha,"[latin, latin pop, rap latina, reggaeton, trap...",81,6674038
9,spotify:artist:3TVXtAsR1Inumwj472S9r4,Drake,"[canadian hip hop, canadian pop, hip hop, rap,...",100,57875139


In [22]:
top_artists_medium.to_csv(r"C:\Users\Ale\Desktop\Data\top_artists_medium.csv")

In [23]:
uri_long, followers_long, genres_long, names_long, popularities_long = get_attributes_from_artists(top_artists_long)

In [24]:
data_top_artists_long = create_df_artists(uri_long, names_long, genres_long, popularities_long, followers_long, columns_artist)
data_top_artists_long

,uri,name,genres,popularity,followers
0,spotify:artist:7f5Zgnp2spUuuzKplmRkt7,Lost Frequencies,"[belgian edm, edm, pop, pop dance, tropical ho...",81,2256994
1,spotify:artist:5K4W6rqBFWDnAN6FQUkS6x,Kanye West,"[chicago rap, rap]",96,15353943
2,spotify:artist:5Pb27ujIyYb33zBqVysBkj,RÜFÜS DU SOL,"[australian electropop, indietronica]",73,838686
3,spotify:artist:1hzfo8twXdOegF3xireCYs,Milky Chance,[german pop],75,1544429
4,spotify:artist:24DO0PijjITGIEWsO8XaPs,Nora En Pure,"[deep house, electra, house, progressive house...",66,289823
5,spotify:artist:23fqKkggKUBHNkbKtXEls4,Kygo,"[edm, pop, pop dance, tropical house]",84,7730716
6,spotify:artist:4WN5naL3ofxrVBgFpguzKo,Rudimental,"[dance pop, edm, house, pop, pop dance, tropic...",74,1298195
7,spotify:artist:73jBynjsVtofjRpdpRAJGk,Dimitri Vegas & Like Mike,"[belgian dance, belgian edm, big room, dance p...",75,3113750
8,spotify:artist:2feDdbD5araYcm6JhFHHw7,Labrinth,"[indie poptimism, pop]",80,1730153
9,spotify:artist:5pKCCKE2ajJHZ9KAiaK11H,Rihanna,"[barbadian pop, dance pop, pop, pop rap, urban...",90,45273762


In [35]:
data_top_artists_long.to_csv(r"C:\Users\Ale\Desktop\Data\top_artists_long.csv")

## Top Tracks

Every track is placed as one element in the list returned. This element is a dictionary, and some of the keys are:
- available markets
- duration_ms
- explicit
- album
    - name
    - release date
    - total tracks
    - artists
        - name
- artist
    - name
- name
- is_local
- popularity
- track_number
- preview url: the song 30-s MP3 preview

In [33]:
columns_tracks = ['track_uri', 'track_name', 'duration_ms', 'explicit', 'artist_name', 'popularity', 
                 'album_name', 'album_release_date', 'album_total_tracks', 'album_track_number', 'available_markets', 
                 'preview_url']

In [34]:
track_uri, track_name, duration, explicit, artist_name, popularity, album_name, album_release_date, album_total_tracks, album_track_number, available_markets, preview_url = get_attributes_from_tracks(top_tracks_long)

In [35]:
data_top_tracks_long = create_df_tracks(track_uri, track_name, duration, explicit, artist_name, popularity, album_name, album_release_date, album_total_tracks, album_track_number, available_markets, preview_url)
data_top_tracks_long

,track_uri,track_name,duration_ms,explicit,artist_name,popularity,album_name,album_release_date,album_total_tracks,album_track_number,available_markets,preview_url
0,spotify:track:19a3JfW8BQwqHWUMbcqSx8,Famous,196040,True,[Kanye West],73,The Life Of Pablo,2016-06-10,20,4,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",https://p.scdn.co/mp3-preview/fb87602292db6f4d...
1,spotify:track:7M0fsXo6rpXOKkveYooBob,Goodbye,256786,False,[Apparat],0,The Devil's Walk,2011,10,4,[],None
2,spotify:track:4WIpEgkpmeVPTBAO4pm5zy,Clouds,257306,False,[Milky Chance],45,Blossom (Deluxe),2017-03-17,20,5,"[AD, AE, AR, AT, BE, BG, BH, BO, BR, CH, CL, C...",https://p.scdn.co/mp3-preview/9a254685de22a411...
3,spotify:track:0Hvvs3VDBQzJ6ksSLA7l4G,Demanding Excellence,210837,False,[Labrinth],46,Euphoria (Original Score from the HBO Series),2019-10-04,26,21,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",https://p.scdn.co/mp3-preview/06802dd73cf9575e...
4,spotify:track:2fZW7ByWe46rGih0FtP2XK,Nothing Left (feat. Will Heard),236682,False,"[Kygo, Will Heard]",51,Cloud Nine,2016-05-13,15,12,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",https://p.scdn.co/mp3-preview/b8ab3ba12d30be95...
5,spotify:track:7zcmIz44ipSUMprZKvEeRO,Free My Mind,160645,False,"[Alok, Rooftime, Dubdogz]",63,Free My Mind,2020-02-21,1,1,[BR],https://p.scdn.co/mp3-preview/dd806df98099bb43...
6,spotify:track:695kvY0b7RypIKs9uiRKp5,"Toast to Our Differences (feat. Shungudzo, Pro...",254773,False,"[Rudimental, Shungudzo, Protoje, Hak Baker]",40,Toast to Our Differences (Deluxe Edition),2019-01-25,16,1,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CH, C...",https://p.scdn.co/mp3-preview/da8f7a6baf19e355...
7,spotify:track:0elXIl26ZrovBULLTrEGK6,Habits of My Heart - EP Version,210639,False,[Jaymes Young],63,Habits of My Heart,2014-09-28,5,1,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",https://p.scdn.co/mp3-preview/d2d47cf2b3629088...
8,spotify:track:4f8Mh5wuWHOsfXtzjrJB3t,I THINK,212013,True,"[Tyler, The Creator]",71,IGOR,2019-05-17,12,3,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",https://p.scdn.co/mp3-preview/02728662c29717a7...
9,spotify:track:42KxOIhF6TIiQWb7hbA8I8,Body Talk (Mamooth),207103,False,"[Moguai, Dimitri Vegas & Like Mike, Julian Per...",34,Body Talk (Mammoth),2014-07-17,2,1,[BR],https://p.scdn.co/mp3-preview/c4109085cb7efcc4...


In [36]:
data_top_tracks_long.to_csv(r"C:\Users\Ale\Desktop\Data\top_tracks_long.csv")

In [37]:
track_uri, track_name, duration, explicit, artist_name, popularity, album_name, album_release_date, album_total_tracks, album_track_number, available_markets, preview_url = get_attributes_from_tracks(top_tracks)

In [38]:
data_top_tracks_medium = create_df_tracks(track_uri, track_name, duration, explicit, artist_name, popularity, album_name, album_release_date, album_total_tracks, album_track_number, available_markets, preview_url)
data_top_tracks_medium

,track_uri,track_name,duration_ms,explicit,artist_name,popularity,album_name,album_release_date,album_total_tracks,album_track_number,available_markets,preview_url
0,spotify:track:0wXuerDYiBnERgIpbb3JBR,Redbone,326933,True,[Childish Gambino],82,"""Awaken, My Love!""",2016-12-02,11,6,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",https://p.scdn.co/mp3-preview/0167089f98ed9b52...
1,spotify:track:24NPziq3dayIMKppeI94Um,Last Night,255706,False,"[Keyshia Cole, Diddy]",62,Just Like You,2007,16,2,"[AD, AR, AT, AU, BE, BG, BO, BR, CH, CL, CO, C...",https://p.scdn.co/mp3-preview/76c132aaba9172f0...
2,spotify:track:6t90Z9XkdsHD8xMxro6KRP,Consideration,161066,True,"[Rihanna, SZA]",68,ANTI (Deluxe),2016-01-28,16,1,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",https://p.scdn.co/mp3-preview/5ed5f5f05f603c7e...
3,spotify:track:2xUsXYIiK18wSnv2SXFGtj,Invisible - Paul Kalkbrenner Remix,457036,False,"[NTO, Paul Kalkbrenner]",61,Invisible (Paul Kalkbrenner Remix),2021-04-14,1,1,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",https://p.scdn.co/mp3-preview/1b78edc8ab5d8bd7...
4,spotify:track:54CipIOeFb1lZYFhHi26lo,Claire,287579,False,"[Anyma, Janus Rasmussen, Delhia De France]",50,Claire,2021-07-16,3,1,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",https://p.scdn.co/mp3-preview/62842f36259c0fb7...
5,spotify:track:5Pt83OUaK3K9C2if4C5JOk,Gotta Have It,140746,True,"[JAY-Z, Kanye West]",66,Watch The Throne (Deluxe),2011-08-08,16,5,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",https://p.scdn.co/mp3-preview/35efe22f72b30584...
6,spotify:track:2wi4d9kls0CWgE4RsghZyR,Rendezvous,260493,False,[RÜFÜS DU SOL],51,Atlas (Light / Dark Deluxe Edition),2013,23,5,"[AE, AR, AT, AU, BH, BO, BR, CA, CH, CL, CO, C...",https://p.scdn.co/mp3-preview/0b50af640b9fda7b...
7,spotify:track:6rGBks2LevDjAMo5KJq88Y,No Goodbye - Extended Version,390920,False,[Paul Kalkbrenner],50,No Goodbye,2019-07-26,2,2,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",https://p.scdn.co/mp3-preview/9e5afcfb2bdd5434...
8,spotify:track:4L7jMAP8UcIe309yQmkdcO,Lost,234093,True,[Frank Ocean],72,channel ORANGE (Explicit Version),2012-07-10,17,11,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CH, C...",https://p.scdn.co/mp3-preview/61a2a0d7b2fb0af2...
9,spotify:track:4slSrbTK1sNK4I1mDYEthf,Rover (feat. DTG),167916,True,"[S1mba, DTG]",72,Rover (feat. DTG),2020-03-04,1,1,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",https://p.scdn.co/mp3-preview/16ce16530b8ef18c...


In [39]:
data_top_tracks_medium.to_csv(r"C:\Users\Ale\Desktop\Data\top_tracks_medium.csv")

## Saved

In [40]:
def get_attributes_from_saved_tracks(data: list):
    track_uri = []
    track_name = []
    added_at = []
    popularity = []
    track_artist = []
    popularity = []
    album_name = []
    album_release_date = []
    duration_ms = []
    preview_url = []

    
    for index in range(len(data)):
        track_uri.append(data[index]['track']['uri'])
        track_name.append(data[index]['track']['name'])
        added_at.append(data[index]['added_at'])
        duration_ms.append(data[index]['track']['duration_ms'])
        track_artist.append([k['name'] for k in list(chain(data[index]['track']['artists']))])
        popularity.append(data[index]['track']['popularity'])
        album_name.append(data[index]['track']['album']['name'])
        album_release_date.append(data[index]['track']['album']['release_date'])
        preview_url.append(data[index]['track']['preview_url'])

    
    return track_uri, track_name, duration_ms, added_at, track_artist, popularity, album_name, album_release_date, preview_url

In [41]:
def create_df_saved_tracks(track_uri: list, track_name: list, duration: list, track_artist: list,
                    popularity: list, album_name: list, album_release_date: list, added_at:list, preview_url: list, 
                           columns_track: list) -> pd.DataFrame:
    
    selected_data = {}
    
    selected_data[columns_track[0]] = track_uri
    selected_data[columns_track[1]] = track_name
    selected_data[columns_track[2]] = track_artist
    selected_data[columns_track[3]] = added_at
    selected_data[columns_track[4]] = duration_ms
    selected_data[columns_track[5]] = popularity
    selected_data[columns_track[6]] = album_name
    selected_data[columns_track[7]] = album_release_date
    selected_data[columns_track[8]] = preview_url
    
    

    return pd.DataFrame.from_dict(selected_data)

In [42]:
columns_saved_tracks = ['track_uri', 'track_name', 'track_artist', 'added_at', 'duration_ms', 'popularity', 'album_name', 'album_release_date', 'preview_url']

In [43]:
track_uri, track_name, duration_ms, added_at, track_artist, popularity, album_name, album_release_date, preview_url = get_attributes_from_saved_tracks(saved_tracks_from_lib)

In [44]:
saved_tracks = create_df_saved_tracks(track_uri, track_name, duration_ms, track_artist, popularity, album_name, album_release_date, added_at, preview_url, columns_saved_tracks)
saved_tracks

,track_uri,track_name,track_artist,added_at,duration_ms,popularity,album_name,album_release_date,preview_url
0,spotify:track:3W8KAzaYIKsCSGj8q8ltul,Father Ocean - Ben Böhmer Remix,"[Monolink, Ben Böhmer]",2021-10-18T16:59:35Z,476192,48,Father Ocean,2018-12-07,https://p.scdn.co/mp3-preview/bd704fc71292e55c...
1,spotify:track:2b6Qg13iooGb1LnBFwrtcH,After Earth,[Ben Böhmer],2021-10-17T15:19:45Z,457301,44,Morning Falls EP,2018-01-05,https://p.scdn.co/mp3-preview/882b30ef349d291f...
2,spotify:track:6mjN72fXeZwLkCDG7rHH3N,After Earth - Edit,[Ben Böhmer],2021-10-17T15:19:16Z,238406,49,Morning Falls EP,2018-01-05,https://p.scdn.co/mp3-preview/94b486ab380f9a20...
3,spotify:track:5kNv1Du0DqYgKkFgXiBRu3,Yours,"[ZHU, Arctic Lake]",2021-10-17T00:08:23Z,257734,46,Yours,2021-04-28,https://p.scdn.co/mp3-preview/783dc0dc84f3120c...
4,spotify:track:3zQpGaH50HeMjxurMZVF6q,Lifetime (with Ty Dolla $ign & 070 Shake),"[Swedish House Mafia, Ty Dolla $ign, 070 Shake]",2021-10-15T01:41:40Z,186805,74,Lifetime,2021-07-19,https://p.scdn.co/mp3-preview/19a577bc12a97e35...
5,spotify:track:4gvrJnKCKIPiacNsWVQwEU,love nwantiti (feat. Dj Yo! & AX'EL) - Remix,"[CKay, DJ Yo, AX'EL]",2021-10-13T16:43:25Z,188368,92,love nwantiti (feat. Dj Yo! & AX'EL) [Remix],2021-09-09,https://p.scdn.co/mp3-preview/fbbecf458216fefa...
6,spotify:track:2g95XDCx4GqcaJPv7TTk8C,Skyscrapers - Hi-Lo Remix,"[Nina Kraviz, HI-LO, Oliver Heldens]",2021-10-08T22:07:11Z,296368,50,Skyscrapers (Hi-Lo Remix),2021-10-01,https://p.scdn.co/mp3-preview/23215101953bd899...
7,spotify:track:3ExRfIXdYwYFjRzsCw7L7S,Dancing People Are Never Wrong (Live In Munich...,"[Jan Blomqvist, The Bianca Story]",2021-10-08T00:21:12Z,283313,22,Disconnected - Live In Munich,2020-08-21,https://p.scdn.co/mp3-preview/c57c49aaf11ad6e3...
8,spotify:track:7hU3IHwjX150XLoTVmjD0q,MONEY,[LISA],2021-10-06T22:07:03Z,168227,90,LALISA,2021-09-10,https://p.scdn.co/mp3-preview/01c7b7f27476d536...
9,spotify:track:6QcU4iwfLjjsrW7zsqyc1D,The Space In Between - Ben Böhmer Remix,"[Jan Blomqvist, Ben Böhmer]",2021-10-04T23:07:58Z,253812,54,The Space In Between (Ben Böhmer Remix),2018-06-29,https://p.scdn.co/mp3-preview/2887027c83d809f9...


In [45]:
saved_tracks.to_csv(r"C:\Users\Ale\Desktop\Data\tracks_saved.csv")